Electricity Forecasting - ARIMA

In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error


elec_df = pd.read_csv(
    'elec/PJM_Load_hourly.csv',
    names=["datetime","energy"],
    dtype={'datetime':'str', 'energy':'float'},
    parse_dates=['datetime'],
    header=0,
    skiprows = 0,
    na_values=['Null']
)

elec_df = elec_df.sort_values(by=['datetime'])
dates = elec_df['datetime']
energy = elec_df['energy']

start_date = '2000-03-1'
end_date = '2000-06-08'
middle_date = '2000-06-01'

all_dates = (dates > start_date)&(dates < end_date)
training_dates = (dates > start_date)&(dates < middle_date)
testing_dates = (dates >= middle_date)&(dates < end_date)

training_data = energy[training_dates]
all_data = energy[all_dates]
dates_list = dates[all_dates]

order = [2, 1, 2]
sorder = [0, 1, 0, 24]
fit = SARIMAX(endog=endog, order=order, seasonal_order=sorder).fit()
ARIMA_forecast = opt_fit.get_forecast(sum(testing_dates))
ARIMA_score = mean_squared_error(energy[testing_dates].values, ARIMA_forecast.predicted_mean, squared=False)


fig, (ax) = plt.subplots(1,1,figsize=(48,27))
ax.set_ylim(0, 50000)
ax.plot(
    dates[all_dates].values,
    energy[all_dates].values,
)
ax.plot(
    dates[testing_dates].values,
    ARIMA_forecast.predicted_mean,
    linestyle = 'dashed')

ax.fill_between(
    dates[testing_dates],
    ARIMA_forecast.predicted_mean-1*ARIMA_forecast.var_pred_mean,
    ARIMA_forecast.predicted_mean+1*ARIMA_forecast.var_pred_mean,
    color='Orange',
    alpha=0.2)
fig.savefig('plots/ARIMA_pred.png')